In [ ]:
import os
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import pickle
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, f1_score, ConfusionMatrixDisplay, roc_curve, RocCurveDisplay
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder

In [ ]:
def default_function(row):
    ## returning 0 for firms with no defaults
    if pd.isnull(row['def_date']):
        return 0
    
    ## checking for a default within 12 months of April of next firm year;
    ## as discussed in the 'When does a firm year end' sidebar,
    ## financial statement data for a given year is not actually available until ~March/April of the next year
    diff_default = row['def_date'] - row['stmt_date']
    
    if diff_default <= datetime.timedelta(days=486) and diff_default > datetime.timedelta(days=120):
        return 1
    else:
        return 0

def reduce_ratio_dimentionality(df, ratio_type, new, preproc_params):
  if new:
    pca = PCA(n_components=1)
    pca.fit(df)
    preproc_params["pca"+ratio_type] = pca
    result = pca.transform(df)
  else:
    pca = preproc_params["pca"+ratio_type]
    result = pca.transform(df)
  return result, preproc_params


def preprocessor(df, preproc_params = {}, data_path="/content/drive/MyDrive/AMLF Project/Data/", new = True):
    cleaned_data = df.copy()
    
    ## dropping column with no non-null values
    cleaned_data = cleaned_data.drop(columns='eqty_corp_family_tot')

    date_cols = ['stmt_date', 'def_date']

    cleaned_data[date_cols[0]] = pd.to_datetime(cleaned_data[date_cols[0]], format='%Y-%m-%d')
    cleaned_data[date_cols[1]] = pd.to_datetime(cleaned_data[date_cols[1]], format='%d/%m/%Y')

    ## filling in NAs for margin_fin and roe by using definition

    cleaned_data.loc[cleaned_data.margin_fin.isna(), 'margin_fin'] = cleaned_data['eqty_tot'] - cleaned_data['asst_current']
    cleaned_data.loc[cleaned_data.roe.isna(), 'roe'] = cleaned_data['prof_operations'] / cleaned_data['eqty_tot']
    
    ## Filling missing data with 0s for HQ_City (categorical) and the fs categories that would be nan for being 0,
    ## based on data definitions.
    ## As well, Converting categorical columns to categorical type

    cleaned_data['HQ_city'] = cleaned_data['HQ_city'].fillna(0).astype('category')
    cleaned_data['legal_struct'] = cleaned_data['legal_struct'].astype('category')
    cleaned_data['ateco_sector'] = cleaned_data['ateco_sector'].astype('category')

    ## calculating inventory from definition
    cleaned_data['inventory'] = cleaned_data['asst_current'] - cleaned_data['cash_and_equiv'] - cleaned_data['AR']
    
    ## building liquidity ratios
    cleaned_data['current_ratio'] = cleaned_data['asst_current'] / cleaned_data['debt_st']
    cleaned_data['quick_ratio'] = (cleaned_data['cash_and_equiv'] + cleaned_data['AR'] 
                                  -  cleaned_data['AP_st'])/ cleaned_data['debt_st']
    cleaned_data['cash_ratio'] = cleaned_data['cash_and_equiv'] / cleaned_data['debt_st']
    cleaned_data['cfo_ratio'] = cleaned_data['cf_operations'] / cleaned_data['debt_st']
    cleaned_data['defensive_interval'] = 365 * (cleaned_data['cash_and_equiv'] + 
                                                cleaned_data['AR'] -  cleaned_data['AP_st']) / cleaned_data['COGS']
    liquidity_ratio_cols = ['current_ratio', 'quick_ratio', 'cash_ratio', 'cfo_ratio', 'defensive_interval']                                            

    ## building activity ratios
    ## get rid of avg_days color 
    cleaned_data['receivables_turnover'] = cleaned_data['rev_operating'] / cleaned_data['AR']
    cleaned_data['inventory_turnover'] = cleaned_data['COGS'] / cleaned_data['inventory']
    cleaned_data['payables_turnover'] = (cleaned_data['COGS'] + cleaned_data['inventory']) / cleaned_data['AP_st']
    cleaned_data['operating_cycle'] = ((365 / cleaned_data['receivables_turnover']) + 
                                       (365 / cleaned_data['inventory_turnover']))
    cleaned_data['net_cash_cycle'] = cleaned_data['operating_cycle'] - (365 / cleaned_data['payables_turnover'])
    cleaned_data['working_capital_turnover'] = cleaned_data['rev_operating'] / cleaned_data['wc_net']
    activity_ratio_cols = ['receivables_turnover', 'inventory_turnover', 'payables_turnover', 'net_cash_cycle', 
                           'working_capital_turnover']

    ## building solvency ratios
    cleaned_data['debt_to_total_assets'] = (cleaned_data['debt_st'] + cleaned_data['debt_lt']) / cleaned_data['asst_tot']
    cleaned_data['debt_to_equity'] = (cleaned_data['debt_st'] + cleaned_data['debt_lt']) / cleaned_data['eqty_tot']
    cleaned_data['financial_leverage'] = cleaned_data['asst_tot'] / cleaned_data['eqty_tot']
    cleaned_data['debt_service_coverage'] = cleaned_data['prof_operations'] / cleaned_data['debt_st']
    cleaned_data['cfo_to_debt'] = cleaned_data['cf_operations'] / (cleaned_data['debt_st'] + cleaned_data['debt_lt'])
    cleaned_data['cfo_to_operating_earnings'] = cleaned_data['cf_operations'] / cleaned_data['prof_operations']
    solvency_ratio_cols =  ['debt_to_total_assets', 'debt_to_equity', 'financial_leverage', 'debt_service_coverage',
                            'cfo_to_debt', 'cfo_to_operating_earnings']

    ## building profitability ratios
    cleaned_data['roic'] = (cleaned_data['prof_operations'] - 
                            cleaned_data['taxes']) / (cleaned_data['wc_net'] + cleaned_data['asst_intang_fixed'] +
                                                      cleaned_data['asst_tang_fixed'] + cleaned_data['asst_fixed_fin'])
    cleaned_data['operating_margin'] = cleaned_data['ebitda'] / cleaned_data['rev_operating']
    cleaned_data['gross_profit_margin'] = (cleaned_data['rev_operating'] - 
                                           cleaned_data['COGS']) / cleaned_data['rev_operating']
    cleaned_data['net_profit_margin_on_sales'] = cleaned_data['profit'] / cleaned_data['rev_operating']
    cleaned_data['cash_roa'] = cleaned_data['cf_operations'] / cleaned_data['asst_tot']
    profitability_ratio_cols = ['roe', 'roic', 'operating_margin', 'gross_profit_margin', 
                                'net_profit_margin_on_sales', 'cash_roa']

    ## Assembling list of ratios
    fs_ratio_cols = liquidity_ratio_cols + activity_ratio_cols + solvency_ratio_cols + profitability_ratio_cols

    ## Replacing infinite values with NaNs
    cleaned_data = cleaned_data.replace([np.inf, -np.inf], np.nan)

    ## imputing missing values for fs_ratio_cols using (train data) industry means
    if new:
      ## creating dict of grouped means
      preproc_params['fs_ratio_group_means'] = {}
      ## iterating through ratio cols
      for ratio_col in fs_ratio_cols:
        ## creating series of grouped means for ratio cols
        preproc_params['fs_ratio_group_means'][ratio_col] = cleaned_data.groupby('ateco_sector')[ratio_col].mean()
        ## filling NAs in each group with the mean
        for group in preproc_params['fs_ratio_group_means'][ratio_col].index:
          cleaned_data.loc[(cleaned_data[ratio_col].isnull()) & (cleaned_data['ateco_sector'] == group), 
                           ratio_col] = preproc_params['fs_ratio_group_means'][ratio_col][group]
    else:
      ## iterating through ratio cols
      for ratio_col in preproc_params['fs_ratio_group_means'].keys():
        ## filling NAs in each group with the mean
        for group in preproc_params['fs_ratio_group_means'][ratio_col].index:
          cleaned_data.loc[(cleaned_data[ratio_col].isnull()) & (cleaned_data['ateco_sector'] == group), 
                           ratio_col] = preproc_params['fs_ratio_group_means'][ratio_col][group]

    ## Doing Principal Component Analysis on each of the ratios to reduce dimentionality
    liquidity_ratio_df = cleaned_data[liquidity_ratio_cols].copy()
    activity_ratio_cols_df = cleaned_data[activity_ratio_cols].copy()
    solvency_ratio_cols_df = cleaned_data[solvency_ratio_cols].copy()
    profitability_ratio_cols_df = cleaned_data[profitability_ratio_cols].copy()

    liquidity_ratio_pca, preproc_params = reduce_ratio_dimentionality(liquidity_ratio_df, "liquidity", new, preproc_params)
    activity_ratio_pca, preproc_params = reduce_ratio_dimentionality(activity_ratio_cols_df, "activity", new, preproc_params)
    solvency_ratio_pca, preproc_params = reduce_ratio_dimentionality(solvency_ratio_cols_df, "solvency", new, preproc_params)
    profitability_ratio_pca, preproc_params = reduce_ratio_dimentionality(profitability_ratio_cols_df, "profitability", new, preproc_params)
    cleaned_data['liquidity_ratio'] = liquidity_ratio_pca
    cleaned_data['activity_ratio'] = activity_ratio_pca
    cleaned_data['solvency_ratio'] = solvency_ratio_pca
    cleaned_data['profitability_ratio'] = profitability_ratio_pca

    ## Assembling list of reduced ratios
    reduced_ratio_cols = ['liquidity_ratio', 'activity_ratio', 'solvency_ratio', 'profitability_ratio']
    
    ## integrating economic data
    italy_econ_data = pd.read_csv(data_path + "italy_economic_data.csv")
    italy_econ_data['Date'] = pd.to_datetime(italy_econ_data['Date'])
    italy_econ_data = italy_econ_data.set_index('Date')
    cleaned_data = pd.merge(cleaned_data, italy_econ_data,
                            left_on='stmt_date', right_index=True)
    italy_econ_cols = list(italy_econ_data.columns)
    
    #categorical variables
    cat_cols = ['legal_struct', 'ateco_sector']
    categorical_columns = []
    
    
    if new:
        one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
        one_hot_encoder.fit(cleaned_data[cat_cols])
        categories = one_hot_encoder.categories_
        dummies = pd.DataFrame(one_hot_encoder.transform(cleaned_data[cat_cols]).toarray())
        categorical_columns=[]
        for j,col in enumerate(cat_cols):
          
          curr_col_categories = categories[j]
          new_columns_names = [col+'_'+str(i) for i in curr_col_categories]
          categorical_columns += new_columns_names
          cleaned_data.drop(col, axis=1, inplace=True)

        dummies.columns = categorical_columns
        cleaned_data = pd.concat([cleaned_data, dummies], axis=1)
        preproc_params['one_hot_encoder'] = one_hot_encoder
    else:
        one_hot_encoder = preproc_params['one_hot_encoder']
        categories = one_hot_encoder.categories_
        dummies = pd.DataFrame(one_hot_encoder.transform(cleaned_data[cat_cols]).toarray())
        categorical_columns=[]
        for j,col in enumerate(cat_cols):
          curr_col_categories = categories[j]
          new_columns_names = [col+'_'+str(i) for i in curr_col_categories]
          categorical_columns += new_columns_names
          cleaned_data.drop(col, axis=1, inplace=True)

        dummies.columns = categorical_columns
        cleaned_data = pd.concat([cleaned_data, dummies], axis=1)

    if new:
      ## building outcome variable using default_function
      cleaned_data['defaulted_within_12_months'] = cleaned_data.apply(lambda x: default_function(x), axis=1)
      del cleaned_data['def_date']

      true_pd = cleaned_data['defaulted_within_12_months'].mean()
      preproc_params['true_pd'] = true_pd

    data_cols = [col for col in cleaned_data.columns if
                 col in reduced_ratio_cols + italy_econ_cols + categorical_columns + ['defaulted_within_12_months']]
    cleaned_data = cleaned_data[data_cols]
    
    num_features = reduced_ratio_cols + italy_econ_cols

    ## Standardizing data
    if new:
      scaler = RobustScaler()
      cleaned_data[num_features] = scaler.fit_transform(cleaned_data[num_features])
      preproc_params['scaler'] = scaler
    else:
      scaler = preproc_params['scaler']
      cleaned_data[num_features] = scaler.transform(cleaned_data[num_features])
      
    return cleaned_data, preproc_params

In [ ]:
def create_calibrator(true_pd, sample_set_pd):
  

  '''
  Function to create a calibrator:
  Arguments:
    true_pd = True probability of default
    sample_set = sample set probability of default
  '''
  def calibrator(model_pd):

    corrected_pd = ((true_pd)*(model_pd - model_pd*sample_set_pd))/(sample_set_pd - model_pd*sample_set_pd + model_pd*true_pd - sample_set_pd*true_pd)

    return corrected_pd

  return calibrator

def estimator(cleaned_data, fitting_algo, calibrator, est_params = {'max_depth':60, 'max_leaf_nodes':1000, 'target_name':'defaulted_within_12_months'}):

    model = fitting_algo(max_depth = est_params['max_depth'])

    X = cleaned_data.loc[:, cleaned_data.columns!=est_params['target_name']]
    y = cleaned_data[est_params['target_name']]
    model.fit(X,y)

    y_pred_proba = model.predict_proba(X)[:,1]
    y_pred = model.predict(X)
    y_pred_proba_calibrated = calibrator(y_pred_proba)


    # print('Before Calibration:')
    # draw_calibration_curve(y, y_pred_proba)
    # print('After Calibration:')
    # draw_calibration_curve(y, y_pred_proba_calibrated)

    print('Area under ROC curve:', roc_auc_score(y, y_pred_proba))
    fpr,tpr,threshs = roc_curve(y, y_pred_proba)
    ConfusionMatrixDisplay.from_predictions(y, y_pred)
    plt.show()   

    plt.plot(fpr, tpr)
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title('ROC Curve')
    plt.show()

    return(model)

def predictor(new_df, model, calibrator):
    # your code here
    predictions = model.predict(new_df)
    predictions_proba = model.predict_proba(new_df)[:,1]
    calibrated_predictions_proba = np.array([calibrator(a) for a in predictions_proba])

    return(predictions,calibrated_predictions_proba)

def test_harness(df, preprocessor=preprocessor, estimator=estimator, predictor=predictor, data_path=None):     
    new_df = df.copy()

    if data_path is None:
      print("Please give Data Path")
      return None

    ## load saved model
    with open(data_path + 'model.pkl' , 'rb') as f:
        model = pickle.load(f)
    ## load save preprocessing parameters
    with open(data_path + 'preproc_params.pkl', 'rb') as fid:
        preproc_params = pickle.load(fid)
    
    ## making sure data is in chronological order
    new_df = new_df.sort_values(preproc_params['year_col'])

    sample_set_pd = preproc_params['sample_set_pd']
    true_pd = preproc_params['true_pd']
    
    calibrator = create_calibrator(true_pd, sample_set_pd)
    ## Preprocessing the test data
    preproc_test_set, test_preproc_params = preprocessor(df, preproc_params, new=False, data_path=data_path)
    
    ## Predicting the default and getting the probabilities
    predictions, pred_proba = predictor(preproc_test_set, model, calibrator)
    
    return pred_proba


In [ ]:
## Kindly install "imbalanced-learn" package
## Eg. conda install -c conda-forge imbalanced-learn
## We would also require python3.7 to run

## Path where all the data is stored
data_path = "./"
## load the test data
test_data = pd.read_csv(data_path+'train.csv')
#del test_data['def_date']

predicted_probabilities = test_harness(test_data, data_path=data_path)